<a href="https://colab.research.google.com/github/talpt/pyton/blob/main/Maestro_Most_Signal.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
!pip install git+https://github.com/rongardF/tvdatafeed  backtesting
!pip install tradingview-screener==2.5.0

import pandas as pd
import numpy as np
from tvDatafeed import TvDatafeed, Interval
from backtesting import Backtest, Strategy
from tradingview_screener import get_all_symbols
import warnings
warnings.simplefilter(action='ignore', category=FutureWarning)

# Kullanıcının zaman dilimini seçmesini sağlayan fonksiyon
def get_interval_choice():
    intervals = {
        '1': ('15 Dakika', Interval.in_15_minute),
        '2': ('30 Dakika', Interval.in_30_minute),
        '3': ('45 Dakika', Interval.in_45_minute),
        '4': ('1 Saat', Interval.in_1_hour),
        '5': ('2 Saat', Interval.in_2_hour),
        '6': ('4 Saat', Interval.in_4_hour),
        '7': ('Günlük', Interval.in_daily),
        '8': ('Haftalık', Interval.in_weekly),
        '9': ('Aylık', Interval.in_monthly)
    }
    print("Lütfen zaman dilimi seçin:")
    for key, value in intervals.items():
        print(f"{key}: {value[0]}")
    choice = input("Seçiminizi yapın (1-9): ")
    return intervals.get(choice, ('1 Saat', Interval.in_1_hour))[1]

#Exponential Moving Average
def ema(series, length):
    """
    Calculate the Exponential Moving Average (EMA) for a given series.
    """
    return series.ewm(span=length, adjust=False).mean()

def MOST(data, percent, n1):
    df = data.copy()  # Working with a copy to avoid modifying the original DataFrame
    percent = percent / 100
    df['EMA'] = ema(df['Close'], length=n1)  # Calculate EMA
    df['tempema'] = 0.0  # Initialize temporary EMA
    df['trend'] = -1  # Initialize trend
    df['MOST'] = 0.0  # Initialize MOST indicator
    df = df.dropna()
    df = df.reset_index()

    for i in range(1, len(df)):
        if df['trend'][i - 1] == 1:
            df.loc[i, 'tempema'] = max(df['tempema'][i - 1], df['EMA'][i])
        elif df['trend'][i - 1] == -1:
            df.loc[i, 'tempema'] = min(df['tempema'][i - 1], df['EMA'][i])

        if df['EMA'][i] >= df['MOST'][i - 1] and df['trend'][i - 1] == 1:
            df.loc[i, 'trend'] = 1
            df.loc[i, 'MOST'] = df['tempema'][i] * (1 - percent)
        elif df['EMA'][i] <= df['MOST'][i - 1] and df['trend'][i - 1] == -1:
            df.loc[i, 'trend'] = -1
            df.loc[i, 'MOST'] = df['tempema'][i] * (1 + percent)
        elif df['EMA'][i] >= df['MOST'][i - 1] and df['trend'][i - 1] == -1:
            df.loc[i, 'trend'] = 1
            df.loc[i, 'MOST'] = df['tempema'][i] * (1 - percent)
        elif df['EMA'][i] <= df['MOST'][i - 1] and df['trend'][i - 1] == 1:
            df.loc[i, 'trend'] = -1
            df.loc[i, 'MOST'] = df['tempema'][i] * (1 + percent)

    df['Entry'] = df['trend'] == 1
    df['Exit'] = df['trend'] == -1
    return df

tv = TvDatafeed()
Hisseler = get_all_symbols(market='turkey')
Hisseler = [symbol.replace('BIST:', '') for symbol in Hisseler]
Hisseler = sorted(Hisseler)

#Raporlama için kullanılacak başlıklar
Titles = ['Hisse Adı', 'Son Fiyat' ,'Kazanma Oranı','Giriş Sinyali', 'Çıkış Sinyali']

df_signals = pd.DataFrame(columns=Titles)

#Backtest için gerekli class yapısı
class Strategy(Strategy):
    def init(self):
        pass
    def next(self):
        if self.data['Entry'] == True and not self.position:
            self.buy()

        elif self.data['Exit'] == True:
            self.position.close()

interval = get_interval_choice()

for i in range(0,len(Hisseler)):
    #print(Hisseler[i])
    try:
        data = tv.get_hist(symbol=Hisseler[i], exchange='BIST', interval=interval, n_bars=500)
        data.rename(columns={'open': 'Open', 'high': 'High', 'low': 'Low', 'close': 'Close', 'volume': 'Volume'}, inplace=True)
        data = data.reset_index()
        MostTrend = MOST(data,2,14)
        MostTrend['datetime'] = pd.to_datetime(MostTrend['datetime'])  # Assuming 'Date' is the name of your datetime column
        MostTrend.set_index('datetime', inplace=True)
        bt = Backtest(MostTrend, Strategy, cash=100000, commission=0.002)
        Stats = bt.run()
        Buy = False
        Sell = False
        Signals = MostTrend.tail(2)
        Signals = Signals.reset_index()
        Buy = Signals.loc[0, 'Entry'] == False and Signals.loc[1, 'Entry'] == True
        Sell = Signals.loc[0, 'Exit'] == False and Signals.loc[1, 'Exit'] == True
        Last_Price = Signals.loc[1, 'Close']
        L1 = [Hisseler[i] ,Last_Price, round(Stats.loc['Win Rate [%]'], 2), str(Buy), str(Sell)]
        df_signals.loc[len(df_signals)] = L1
        print(L1)
    except:
        pass

df_True = df_signals[(df_signals['Giriş Sinyali'] == 'True')]
print(df_True)


  Cloning https://github.com/rongardF/tvdatafeed to /tmp/pip-req-build-3ypx2t6l
  Running command git clone --filter=blob:none --quiet https://github.com/rongardF/tvdatafeed /tmp/pip-req-build-3ypx2t6l
  Resolved https://github.com/rongardF/tvdatafeed to commit e6f6aaa7de439ac6e454d9b26d2760ded8dc4923
  Preparing metadata (setup.py) ... done
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 175.5/175.5 kB 3.1 MB/s eta 0:00:00
  Preparing metadata (setup.py) ... done
  Created wheel for tvdatafeed: filename=tvdatafeed-2.1.0-py3-none-any.whl size=17533 sha256=574b2d59960dae17cc32ae81983361cb9672ce151cd91feba79ea2aa4ee73246
  Stored in directory: /tmp/pip-ephem-wheel-cache-8nom_4c0/wheels/0a/ba/99/b27476fd1e4caf0dd70445cdc6798195d3b90005a1501a12f7
  Created wheel for backtesting: filename=Backtesting-0.3.3-py3-none-any.whl size=173912 sha256=fcf8223928a673d919e0399358ec31480fdf9e4aef0a9c78168efaee2294e059
  Stored in directory: /root/.cache/pip/wheels/2c/56/19/bf7ee5e164aa99a976e3f64841c83b5a

/usr/local/lib/python3.11/dist-packages/backtesting/_plotting.py:50: UserWarning: Jupyter Notebook detected. Setting Bokeh output to notebook. This may not work in Jupyter clients without JavaScript support (e.g. PyCharm, Spyder IDE). Reset with `backtesting.set_bokeh_output(notebook=False)`.
  warnings.warn('Jupyter Notebook detected. '


Lütfen zaman dilimi seçin:
1: 15 Dakika
2: 30 Dakika
3: 45 Dakika
4: 1 Saat
5: 2 Saat
6: 4 Saat
7: Günlük
8: Haftalık
9: Aylık
Seçiminizi yapın (1-9): 7
['A1CAP', 25.06, 22.22, 'False', 'False']
['ACSEL', 117.7, 44.44, 'False', 'False']
['ADEL', 32.98, 18.18, 'False', 'False']
['ADESE', 2.19, 33.33, 'False', 'False']
['ADGYO', 31.04, 42.86, 'False', 'False']
['AEFES', 163.3, 57.14, 'False', 'False']
['AFYON', 14.83, 45.45, 'False', 'False']
['AGESA', 138.9, 30.0, 'False', 'False']
['AGHOL', 299.75, 55.56, 'False', 'False']
['AGROT', 10.18, 40.0, 'False', 'False']
['AGYO', 7.23, 18.18, 'False', 'False']
['AHGAZ', 20.16, 57.14, 'False', 'False']
['AHSGY', 21.28, 50.0, 'False', 'False']
['AKBNK', 63.2, 30.77, 'False', 'False']
['AKCNS', 181.4, 50.0, 'False', 'False']
['AKENR', 13.15, 14.29, 'False', 'False']
['AKFGY', 2.08, 30.0, 'False', 'False']
['AKFYE', 19.35, 75.0, 'False', 'False']
['AKGRT', 7.42, 50.0, 'False', 'False']
['AKMGY', 217.7, 40.0, 'False', 'False']
['AKSEN', 39.7, 44.44